In [1]:
from langchain_community.document_loaders import YoutubeLoader

/home/linux/Documents/learn-ml/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=5c17kgyZnWI", add_video_info=False
)

In [3]:
loader.load()

[Document(metadata={'source': '5c17kgyZnWI'}, page_content="Hello everyone, we will continue the topic about on HANA and currently we are covering various things using Eclipse and in the last video we started with transport organizer using Eclipse. So as a part of that what are the new learnings how you can open transport organizer through eclipse you can open by this particular button or you can simply go to window in the window yes suppose if you will go to window we have a option of suppose I'll click system is slow yes I'll go to window again we'll simply Go to show view. Whenever you will go to show view you will go for other and in the other you have a appab. In the abab you have transport organizer. So this is first learning how to open transport organizer through eclipse. Then after that yes how you can release a request through transport organizer of eclipse. That is a new learning. After that yes how you can check the version how you can check this particular program is into 

In [4]:
import pandas as pd

In [5]:
df = pd.read_json("./rahul_mehta_links.json")

In [6]:
df

,0
0,https://www.youtube.com/watch?v=9ivUnuzOXuw
1,https://www.youtube.com/watch?v=5c17kgyZnWI
2,https://www.youtube.com/watch?v=8ZFv_HanIf8
3,https://www.youtube.com/watch?v=JJK0MpFtdwE
4,https://www.youtube.com/watch?v=jVA1A9FdcIw
...,...
16907,https://www.youtube.com/watch?v=sRRgMZ6ps68
16908,https://www.youtube.com/watch?v=5HIAa_mlxB4
16909,https://www.youtube.com/watch?v=ArGilh_hCtQ
16910,https://www.youtube.com/watch?v=roMcnToz0ao


In [18]:
set_of_ten = df[10:20]
set_of_ten

,0
10,https://www.youtube.com/watch?v=l9KtTAUAM20
11,https://www.youtube.com/watch?v=mKV7eBa4Hn4
12,https://www.youtube.com/watch?v=BN5GhBN6Jw4
13,https://www.youtube.com/watch?v=HbOyVekz46g
14,https://www.youtube.com/watch?v=3kK8c9d2vXY
15,https://www.youtube.com/watch?v=zNnBO3eGtfs
16,https://www.youtube.com/watch?v=YM7ax0eaeUM
17,https://www.youtube.com/watch?v=HJeaIHrHrHg
18,https://www.youtube.com/watch?v=lS9pZ6xOKQE
19,https://www.youtube.com/watch?v=dPzsTPuKWEM


In [19]:
rahul_mehta_content = []

In [ ]:
import time

for index, row in set_of_ten.iterrows():
    video_url = row[0]
    loader = YoutubeLoader.from_youtube_url(video_url, add_video_info=False)
    
    try:
        document = loader.load()
        rahul_mehta_content.append(document)
    except Exception as e:
        print(f"Failed for {video_url}: {e}")
    
    time.sleep(3)

11
12
13
14
15
16
17
18
19
20


In [30]:
import subprocess
import os
import re

# Set this path to wherever you saved the exported cookies file
COOKIES_PATH = os.path.expanduser("./cookies.txt")


def get_transcript_via_ytdlp(video_url, output_dir="/tmp"):
    output_template = os.path.join(output_dir, "%(id)s")

    result = subprocess.run([
        "yt-dlp",
        "--cookies", COOKIES_PATH,        # use the exported cookie file instead
        "--write-auto-sub",
        "--sub-lang", "en",
        "--sub-format", "vtt",
        "--skip-download",
        "-o", output_template,
        video_url
    ], capture_output=True, text=True)

    # Find the .vtt file that yt-dlp wrote to disk
    vtt_files = [f for f in os.listdir(output_dir) if f.endswith(".en.vtt")]

    if not vtt_files:
        print("No subtitle file found. yt-dlp said:")
        print(result.stderr[-800:])  # print errors to help debug
        return None

    vtt_path = os.path.join(output_dir, sorted(vtt_files)[-1])

    with open(vtt_path, "r", encoding="utf-8") as f:
        raw_vtt = f.read()

    os.remove(vtt_path)  # clean up after reading
    return parse_vtt_to_text(raw_vtt)


def parse_vtt_to_text(vtt_content):
    lines = vtt_content.split("\n")
    text_lines = []

    for line in lines:
        line = line.strip()
        if (not line
                or line.startswith("WEBVTT")
                or line.startswith("NOTE")
                or "-->" in line
                or re.match(r"^\d+$", line)):
            continue
        line = re.sub(r"<[^>]+>", "", line)  # strip VTT inline timing tags
        if line:
            text_lines.append(line)

    # VTT files often repeat lines as captions "roll", so deduplicate adjacent ones
    deduped = [text_lines[0]] if text_lines else []
    for line in text_lines[1:]:
        if line != deduped[-1]:
            deduped.append(line)

    return " ".join(deduped)

In [ ]:
import time
from langchain_core.documents import Document

rahul_mehta_content = []

for index, row in set_of_ten.iterrows():
    video_url = row[0]
    
    try:
        transcript_text = get_transcript_via_ytdlp(video_url)
        
        if transcript_text:
            # Wrap in a LangChain Document so downstream code stays unchanged
            doc = Document(
                page_content=transcript_text,
                metadata={"source": video_url}
            )
            rahul_mehta_content.append([doc])  # list-wrapped to match loader.load() output
            print(f"✓ Got transcript for {video_url} ({len(transcript_text)} chars)")
        else:
            print(f"✗ No transcript found for {video_url}")
            
    except Exception as e:
        print(f"✗ Error for {video_url}: {e}")
    
    # Be respectful — give YouTube a breather between requests
    time.sleep(3)

In [31]:
test_result = subprocess.run([
    "yt-dlp",
    "--cookies", COOKIES_PATH,
    "--skip-download",
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
], capture_output=True, text=True)

print(test_result.stdout[-500:])
print("ERRORS:", test_result.stderr[-300:])

[youtube] Extracting URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
[youtube] dQw4w9WgXcQ: Downloading webpage
[youtube] dQw4w9WgXcQ: Downloading tv downgraded player API JSON
[youtube] dQw4w9WgXcQ: Downloading web safari player API JSON

ERRORS: gs presented before this message. For more details, refer to  https://github.com/yt-dlp/yt-dlp/wiki/EJS
ERROR: [youtube] dQw4w9WgXcQ: Requested format is not available. Use --list-formats for a list of available formats



In [ ]:
import time

for index, row in set_of_ten.iterrows():
    video_url = row[0]
    loader = YoutubeLoader.from_youtube_url(video_url, add_video_info=False)
    
    try:
        document = loader.load()
        rahul_mehta_content.append(document)
    except Exception as e:
        print(f"Failed for {video_url}: {e}")
    
    time.sleep(3)

KeyboardInterrupt: 

In [ ]:
import json

In [14]:
with open("rahul_mehta_content.json", "w") as outfile:
    json.dump(rahul_mehta_content, outfile, default=lambda x: x.__dict__)

In [26]:
content_df = pd.read_json("./rahul_mehta_content.json")
content_df

,0
0,"{'id': None, 'metadata': {'source': '9ivUnuzOX..."
1,"{'id': None, 'metadata': {'source': '5c17kgyZn..."
2,"{'id': None, 'metadata': {'source': '8ZFv_HanI..."
3,"{'id': None, 'metadata': {'source': 'JJK0MpFtd..."
4,"{'id': None, 'metadata': {'source': 'jVA1A9Fdc..."
5,"{'id': None, 'metadata': {'source': 'Hf6nUE-V0..."
6,"{'id': None, 'metadata': {'source': 'pqpXycGot..."
7,"{'id': None, 'metadata': {'source': 'UD_FbMDB4..."
8,"{'id': None, 'metadata': {'source': 'X_UWtFoeg..."
9,"{'id': None, 'metadata': {'source': 'W-JMmgIUi..."
